# IMPORTS

In [1]:
import pandas as pd
import numpy as np

In [3]:
df= pd.read_csv(r"BTC-Dataset-USDM-2024-04-01 - 2024-04-17 - 5M.csv")

In [6]:
df

,Unnamed: 0,date_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,volume_true,volume_false,vdiff(false-true),side,original_volume,last_fill_volume,accumulated_fill_volume
0,0,2024-04-01 00:00:00,71363.0,71388.0,71200.0,71323.7,1551.338,1.106063e+08,21436,576.616,4.111328e+07,6.949298e+07,4.111328e+07,-2.837970e+07,SELL,5403752.0,5403752.0,5403752.0
1,1,2024-04-01 00:05:00,71323.6,71339.7,71193.2,71295.3,630.072,4.489739e+07,11151,201.764,1.437777e+07,3.051962e+07,1.437777e+07,-1.614185e+07,0,0.0,0.0,0.0
2,2,2024-04-01 00:10:00,71295.2,71295.3,71204.9,71230.0,451.267,3.214895e+07,6729,167.854,1.195777e+07,2.019117e+07,1.195777e+07,-8.233402e+06,0,0.0,0.0,0.0
3,3,2024-04-01 00:15:00,71230.0,71286.6,71182.5,71286.6,623.674,4.442764e+07,8418,330.932,2.357946e+07,2.084818e+07,2.357946e+07,2.731287e+06,0,0.0,0.0,0.0
4,4,2024-04-01 00:20:00,71286.5,71286.9,71229.7,71261.4,202.889,1.445906e+07,5048,81.038,5.775295e+06,8.683766e+06,5.775295e+06,-2.908472e+06,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885,4885,2024-04-17 23:05:00,61577.4,61743.7,61529.6,61676.8,643.289,3.966970e+07,10868,359.219,2.215607e+07,1.751363e+07,2.215607e+07,4.642434e+06,BUY,6319144.8,2849810.4,6319144.8
4886,4886,2024-04-17 23:10:00,61676.6,61700.0,61554.1,61554.1,498.433,3.071231e+07,8083,117.106,7.216019e+06,2.352097e+07,7.216019e+06,-1.630495e+07,0,0.0,0.0,0.0
4887,4887,2024-04-17 23:15:00,61554.1,61554.1,61363.6,61366.4,846.756,5.202591e+07,13121,227.749,1.399381e+07,3.800741e+07,1.399381e+07,-2.401360e+07,0,0.0,0.0,0.0
4888,4888,2024-04-17 23:20:00,61366.4,61397.7,61345.0,61365.5,479.846,2.944704e+07,8809,198.138,1.215926e+07,1.728778e+07,1.215926e+07,-5.128522e+06,0,0.0,0.0,0.0


# Define Price Based Indicators

In [4]:
# Function to calculate Exponential Moving Averages (EMAs)
def calculate_ema(data, periods=[12, 26, 50, 100, 200, 500]):
    for period in periods:
        ema_name = f'EMA_{period}'
        data[ema_name] = data['close'].ewm(span=period, adjust=False).mean()
    return data

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Function to calculate Stochastic Oscillator
def calculate_stochastic(data, k_period=14, d_period=3):
    lowest_low = data['low'].rolling(window=k_period).min()
    highest_high = data['high'].rolling(window=k_period).max()
    data['stochastic_%K'] = ((data['close'] - lowest_low) / (highest_high - lowest_low)) * 100
    data['stochastic_%D'] = data['stochastic_%K'].rolling(window=d_period).mean()
    return data

# Function to calculate Moving Average Convergence Divergence (MACD)
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    short_ema = data['close'].ewm(span=short_window, adjust=False).mean()
    long_ema = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = short_ema - long_ema
    data['MACD_signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data['close'].rolling(window=window).mean()
    rolling_std = data['close'].rolling(window=window).std()
    data['Bollinger_Band_Upper'] = rolling_mean + (rolling_std * num_std)
    data['Bollinger_Band_Lower'] = rolling_mean - (rolling_std * num_std)
    return data

# Add The Indicators to Dataframe

In [7]:
# Calculate RSI
df['RSI'] = calculate_rsi(df)

# Calculate Stochastic Oscillator
df = calculate_stochastic(df)

# Calculate EMAs
df = calculate_ema(df)

# Calculate MACD
df = calculate_macd(df)

# Calculate Bollinger Bands
df = calculate_bollinger_bands(df)


In [11]:
# Display the updated DataFrame
df.tail(5)

,Unnamed: 0,date_time,open,high,low,close,volume,quote_volume,count,taker_buy_volume,...,EMA_12,EMA_26,EMA_50,EMA_100,EMA_200,EMA_500,MACD,MACD_signal,Bollinger_Band_Upper,Bollinger_Band_Lower
4885,4885,2024-04-17 23:05:00,61577.4,61743.7,61529.6,61676.8,643.289,3.966970e+07,10868,359.219,...,61503.075030,61392.921595,61311.984491,61403.070173,61830.899388,62625.473167,110.153435,80.264697,61715.955691,61034.764309
4886,4886,2024-04-17 23:10:00,61676.6,61700.0,61554.1,61554.1,498.433,3.071231e+07,8083,117.106,...,61510.925025,61404.860736,61321.479217,61406.060863,61828.145165,62621.196228,106.064289,85.424615,61735.648779,61041.421221
4887,4887,2024-04-17 23:15:00,61554.1,61554.1,61363.6,61366.4,846.756,5.202591e+07,13121,227.749,...,61488.690406,61402.011793,61323.240816,61405.275499,61823.550686,62616.187062,86.678613,85.675415,61734.064574,61057.645426
4888,4888,2024-04-17 23:20:00,61366.4,61397.7,61345.0,61365.5,479.846,2.944704e+07,8809,198.138,...,61469.738036,61399.307216,61324.898039,61404.487865,61818.992968,62611.194299,70.430820,82.626496,61717.870591,61100.479409
4889,4889,2024-04-17 23:25:00,61365.5,61365.6,61218.8,61248.0,1093.849,6.701037e+07,14027,386.775,...,61435.624492,61388.099274,61321.882430,61401.389096,61813.311446,62605.752406,47.525218,75.606240,61717.027521,61102.622479


# Clean The Dataframe & Remove Unneccassary Columns

In [12]:
df['close_pct'] = df['close'].pct_change() * 100
df = df.drop(columns=['Unnamed: 0', 'side', 'high','low','close','open','taker_buy_volume','last_fill_volume', 'accumulated_fill_volume','volume','taker_buy_quote_volume'])
df.dropna(inplace=True)
df.set_index('date_time', inplace=True)

In [13]:
df

,quote_volume,count,volume_true,volume_false,vdiff(false-true),original_volume,RSI,stochastic_%K,stochastic_%D,EMA_12,EMA_26,EMA_50,EMA_100,EMA_200,EMA_500,MACD,MACD_signal,Bollinger_Band_Upper,Bollinger_Band_Lower,close_pct
date_time,,,,,,,,,,,,,,,,,,,,
2024-04-01 01:35:00,4.994263e+07,12218,3.093844e+07,1.900398e+07,-1.193446e+07,27715458.4,40.161655,2.864583,31.186410,71102.155534,71165.873481,71220.770789,71264.726168,71292.016783,71310.452326,-63.717947,-51.240846,71360.361741,70945.738259,-0.327303
2024-04-01 01:40:00,5.211234e+07,11452,2.628917e+07,2.582316e+07,-4.660128e+05,3531960.0,39.341375,9.650538,26.264383,71074.731606,71147.949519,71209.128797,71257.977135,71288.353930,71308.909203,-73.217914,-55.636259,71348.047171,70918.072829,-0.000564
2024-04-01 01:45:00,5.469217e+07,10759,2.879846e+07,2.589371e+07,-2.904755e+06,0.0,43.214088,17.920585,10.145235,71054.895974,71132.975481,71198.802178,71251.795410,71284.945433,71307.459665,-78.079507,-60.124909,71331.859448,70899.310552,0.030878
2024-04-01 01:50:00,3.390074e+07,8694,1.959414e+07,1.430660e+07,-5.287547e+06,0.0,45.115026,8.292808,11.954644,71031.327363,71115.843964,71187.151112,71244.862828,71281.132046,71305.839866,-84.516601,-65.003247,71328.331622,70870.008378,-0.062160
2024-04-01 01:55:00,2.427866e+07,6167,1.300361e+07,1.127506e+07,-1.728551e+06,0.0,36.746793,6.532037,10.915143,71008.830845,71098.751818,71175.305971,71237.738811,71277.191428,71304.160266,-89.920973,-69.986792,71309.464191,70848.725809,-0.023413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-17 23:05:00,3.966970e+07,10868,1.751363e+07,2.215607e+07,4.642434e+06,6319144.8,73.416518,88.980399,87.237756,61503.075030,61392.921595,61311.984491,61403.070173,61830.899388,62625.473167,110.153435,80.264697,61715.955691,61034.764309,0.161260
2024-04-17 23:10:00,3.071231e+07,8083,2.352097e+07,7.216019e+06,-1.630495e+07,0.0,70.252445,68.769560,78.724669,61510.925025,61404.860736,61321.479217,61406.060863,61828.145165,62621.196228,106.064289,85.424615,61735.648779,61041.421221,-0.198940
2024-04-17 23:15:00,5.202591e+07,13121,3.800741e+07,1.399381e+07,-2.401360e+07,0.0,54.311572,34.314067,64.021342,61488.690406,61402.011793,61323.240816,61405.275499,61823.550686,62616.187062,86.678613,85.675415,61734.064574,61057.645426,-0.304935
